# 캠페인과 상호 작용 <a class="anchor" id="top"></a>

이 노트북에서는 Amazon Personalize에서 캠페인을 배포하고 상호 작용합니다.

1. [소개](#intro)
1. [캠페인 생성](#create)
1. [캠페인과 상호 작용](#interact)
1. [배치 추천](#batch)
1. [마무리](#wrapup)

## 소개 <a class="anchor" id="intro"></a>
[맨 위로 이동](#top)

이 시점에는 몇 개의 솔루션과 각 솔루션의 버전이 하나 만들어져 있습니다. 일단 솔루션 버전이 생성되면 해당 버전으로부터 추천을 받고 전반적인 동작을 파악할 수 있습니다.

이 노트북에서는 이전 노트북의 각 솔루션 버전을 개별 캠페인에 배포하는 것부터 시작합니다. 캠페인이 활성화되면 추천을 쿼리하기 위한 리소스와 사용자에게 좀 더 읽기 쉬운 출력으로 요약하는 헬퍼 함수가 제공됩니다.

Amazon Personalize를 사용하는 고객과 마찬가지로, 헬퍼 함수를 데이터 입력 파일의 구조에 맞게 수정하여 추가 렌더링이 계속 작동하도록 할 수 있습니다.

시작하려면 라이브러리를 가져오고 이전 노트북의 값을 로드하고 SDK를 로드해야 합니다.

In [1]:
import time
from time import sleep
import json
from datetime import datetime
import uuid
import random

import boto3
import pandas as pd

In [2]:
%store -r

In [3]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

## 캠페인과 상호 작용 <a class="anchor" id="interact"></a>
[맨 위로 이동](#top)

이제 모든 캠페인이 배포되고 활성화되었으므로 API 호출을 통해 추천을 받을 수 있습니다. 각 캠페인은 약간 다른 방식으로 작동하는 다른 레시피를 기반으로, 서로 다른 사용 사례를 지원합니다. 각 캠페인은 이전 노트북과 다른 순서로 진행되며, 복잡성이 발생할 수 있는 경우 오름차순(즉, 가장 간단한 것부터)에 따라 처리됩니다.

먼저 Personalize 캠페인이 반환한 결과를 이해할 수 있도록 지원 함수를 만들어 보겠습니다. Personalize는 `item_id`만 반환합니다. 이 함수는 데이터를 압축하기에 좋지만, 사용자가 읽을 수 있는 노트북 결과를 얻으려면 데이터베이스나 조회 테이블을 쿼리해야 합니다. 헬퍼 함수를 생성하여 FM 데이터 세트에서 사용자가 읽을 수 있는 결과를 반환하겠습니다.

먼저 조회 테이블에 사용할 수 있는 데이터 세트를 로드합니다.

In [4]:
# Create a dataframe for the items by reading in the correct source CSV
items_df = pd.read_csv(dataset_dir + '/movies.csv', sep=',', usecols=[0,1], encoding='latin-1', dtype={'movieId': "object", 'title': "str"},index_col=0)

# Render some sample data
items_df.head(5)

title
movieId                                    
1                          Toy Story (1995)
2                            Jumanji (1995)
3                   Grumpier Old Men (1995)
4                  Waiting to Exhale (1995)
5        Father of the Bride Part II (1995)

ID 열을 인덱스 열로 정의하면 ID를 쿼리하는 것만으로 아티스트를 반환할 수 있습니다.

In [5]:
movie_id_example = 589
title = items_df.loc[movie_id_example]['title']
print(title)

Terminator 2: Judgment Day (1991)


그렇게 나쁘지는 않지만, 코드의 모든 곳에서 이 작업을 반복하면 더 복잡해집니다. 그래서 아래의 함수가 이를 정리해줄 것입니다.

In [6]:
def get_movie_by_id(movie_id, movie_df=items_df):
    """
    This takes in an artist_id from Personalize so it will be a string,
    converts it to an int, and then does a lookup in a default or specified
    dataframe.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return movie_df.loc[int(movie_id)]['title']
    except:
        return "Error obtaining title"

이제 오류 캐칭 기능을 확인하기 위해 몇 가지 간단한 값을 테스트해 보겠습니다.

In [7]:
# A known good id (The Princess Bride)
print(get_movie_by_id(movie_id="1197"))
# A bad type of value
print(get_movie_by_id(movie_id="987.9393939"))
# Really bad values
print(get_movie_by_id(movie_id="Steve"))

Princess Bride, The (1987)
Error obtaining title
Error obtaining title


좋습니다. 이제 결과를 렌더링할 수 있습니다. 

### SIMS

SIMS는 입력으로 항목만 요구하며 입력 항목과 유사한 방식으로 사용자가 상호 작용하는 항목을 반환합니다. 이 예의 경우 그 항목은 바로 영화입니다. 

아래의 셀은 SIMS로부터 추천을 받고 결과를 렌더링하는 작업을 처리합니다. 이 노트북의 앞부분에서 살펴본 첫 번째 항목(터미네이터 2: 심판의 날)에 대한 추천을 살펴보겠습니다.

In [8]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = sims_campaign_arn,
    itemId = str(589),
)

In [9]:
item_list = get_recommendations_response['itemList']
for item in item_list:
    print(get_movie_by_id(movie_id=item['itemId']))

Jurassic Park (1993)
Braveheart (1995)
Terminator, The (1984)
Fugitive, The (1993)
Speed (1994)
Crimson Tide (1995)
GoldenEye (1995)
Batman (1989)
Clear and Present Danger (1994)
True Lies (1994)
Mask, The (1994)
Die Hard: With a Vengeance (1995)
In the Line of Fire (1993)
Lion King, The (1994)
Ghost (1990)
Forrest Gump (1994)
Apollo 13 (1995)
Cliffhanger (1993)
Star Trek: Generations (1994)
Firm, The (1993)
Die Hard (1988)
Seven (a.k.a. Se7en) (1995)
Indiana Jones and the Last Crusade (1989)
Mission: Impossible (1996)
Mrs. Doubtfire (1993)


축하합니다! 첫 번째 추천 목록을 받았습니다. 이 목록도 괜찮지만, 아티스트 샘플 컬렉션에 대한 추천을 멋진 데이터 프레임으로 렌더링하는 것이 좋습니다. 다시 한 번 헬퍼 함수를 만들어 보겠습니다.

In [10]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df(recommendations_df, movie_ID):
    # Get the movie name
    movie_name = get_movie_by_id(movie_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = sims_campaign_arn,
        itemId = str(movie_ID),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

이제 여러 편의 영화로 헬퍼 함수를 테스트해 보겠습니다. SIMS 캠페인을 테스트하기 위해 데이터 세트에서 몇 가지 데이터를 추출해 보겠습니다. 데이터 프레임에서 임의의 영화 5편을 선택합니다.

참고: 유사한 제목을 표시하므로 나열된 일부 영화를 인식할 때까지 샘플을 다시 실행할 수 있습니다.

In [16]:
samples = items_df.sample(5)
samples

title
movieId                              
1911              Dr. Dolittle (1998)
8451         Blackboard Jungle (1955)
971      Cat on a Hot Tin Roof (1958)
3099                   Shampoo (1975)
2944          Dirty Dozen, The (1967)

In [17]:
sims_recommendations_df = pd.DataFrame()
movies = samples.index.tolist()

for movie in movies:
    sims_recommendations_df = get_new_recommendations_df(sims_recommendations_df, movie)

sims_recommendations_df

Dr. Dolittle (1998)  \
0                         A Bad Moms Christmas (2017)   
1                 Jim & Andy: The Great Beyond (2017)   
2                        O.J.: Made in America (2016)   
3                          Gaga: Five Foot Two (2017)   
4                                Ice Guardians (2016)   
5                              Kurt & Courtney (1998)   
6                 Kevin Hart: Laugh at My Pain (2011)   
7   Nobody Speak: Hulk Hogan, Gawker and Trials of...   
8                            Making a Murderer (2015)   
9                                   Blue Crush (2002)   
10                           When We First Met (2018)   
11  Armour of God II: Operation Condor (Operation ...   
12                             Honest Liar, An (2014)   
13                               Super High Me (2007)   
14  Going Clear: Scientology and the Prison of Bel...   
15                          Drop Dead Gorgeous (1999)   
16                                        Goon (2011)   
17                                Multiplicity (1996)   
18      Romy and Michele's High School Reunion (1997)   
19                               Runaway Bride (1999)   
20                                  Ride Along (2014)   
21                                       Shaft (2000)   
22                               Little Buddha (1993)   
23            Goods: Live Hard, Sell Hard, The (2009)   
24                                      Gifted (2017)   

                             Blackboard Jungle (1955)  \
0                    Shawshank Redemption, The (1994)   
1                                 Forrest Gump (1994)   
2                                 Pulp Fiction (1994)   
3                    Silence of the Lambs, The (1991)   
4                                   Braveheart (1995)   
5                                  Matrix, The (1999)   
6                             Schindler's List (1993)   
7           Star Wars: Episode IV - A New Hope (1977)   
8                                Jurassic Park (1993)   
9                   Terminator 2: Judgment Day (1991)   
10                                   Apollo 13 (1995)   
11                               Fugitive, The (1993)   
12                                  Fight Club (1999)   
13                                   Toy Story (1995)   
14                        Seven (a.k.a. Se7en) (1995)   
15                         Usual Suspects, The (1995)   
16                             American Beauty (1999)   
17  Raiders of the Lost Ark (Indiana Jones and the...   
18  Lord of the Rings: The Fellowship of the Ring,...   
19                                       Fargo (1996)   
20  Lord of the Rings: The Return of the King, The...   
21                          Dances with Wolves (1990)   
22               Independence Day (a.k.a. ID4) (1996)   
23      Lord of the Rings: The Two Towers, The (2002)   
24                              Godfather, The (1972)   

                         Cat on a Hot Tin Roof (1958)  \
0                     Thomas Crown Affair, The (1968)   
1                         Ox-Bow Incident, The (1943)   
2                               Dolce Vita, La (1960)   
3                        Long, Hot Summer, The (1958)   
4                           Von Ryan's Express (1965)   
5                    Requiem for a Heavyweight (1962)   
6                        Suddenly, Last Summer (1959)   
7                      Run of the Country, The (1995)   
8                            Stand and Deliver (1988)   
9                                 Turtle Diary (1985)   
10                                Mission, The (1986)   
11                                Paris, Texas (1984)   
12                     The Diary of Anne Frank (1959)   
13                            Princess Caraboo (1994)   
14                                     Tin Men (1987)   
15    Cook the Thief His Wife & Her Lover, The (1989)   
16                                       Marty (1955)   
17                               Meet John

많은 항목이 동일해 보이지만 모든 항목이 동일하지는 않을 수 있습니다(이 경우 상호 작용 횟수가 적을 가능성이 더 높으며, 이는 movielens 소규모 데이터 세트에서 더 일반적). 이는 솔루션 버전을 평가할 때 평가 지표에만 의존해서는 안 된다는 것을 보여줍니다. 그러면 이런 일이 일어날 때, 어떻게 하면 결과를 개선할 수 있을까요?

이쯤에서 개인화 레시피의 하이퍼파라미터에 대해 생각해 보는 것이 좋습니다. SIMS 레시피에는 `popularity_discount_factor` 하이퍼파라미터가 있습니다([설명서](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-sims.html) 참조). 이 하이퍼파라미터를 활용하면 결과에 나타나는 뉘앙스를 제어할 수 있습니다. 이 파라미터와 그 동작은 사용자가 접하는 모든 데이터 세트에 고유하며 비즈니스 목표에 따라 달라집니다. 결과에 만족할 때까지 이 하이퍼파라미터 값을 반복하거나, Personalize의 하이퍼파라미터 최적화(HPO) 기능을 활용하여 시작할 수 있습니다. 하이퍼파라미터 및 HPO 튜닝에 대한 자세한 내용은 [설명서](https://docs.aws.amazon.com/personalize/latest/dg/customizing-solution-config-hpo.html)를 참조하세요.

### 사용자 개인화

HRNN은 Amazon Personalize에서 제공하는 고급 알고리즘 중 하나입니다. 이 알고리즘은 특정 사용자의 과거 동작을 기준으로 항목을 개인화할 수 있도록 지원하며, 다시 훈련하지 않고도 사용자에 대한 추천을 변경하기 위해 실시간 이벤트를 가져올 수 있습니다.

HRNN은 사용자의 샘플을 사용하므로, 샘플로 사용할 데이터를 로드하고 임의의 사용자 3명을 선택합니다. Movielens는 사용자 데이터를 포함하지 않기 때문에 데이터 세트의 사용자 ID 범위에서 3개의 난수를 선택합니다.

In [13]:
if not USE_FULL_MOVIELENS:
    users = random.sample(range(1, 600), 3)
else:
    users = random.sample(range(1, 162000), 3)
users

[115, 518, 252]

이제 위에서 선택한 임의의 사용자 3명을 위한 추천을 렌더링하겠습니다. 그런 다음 개인별 순위로 넘어가기 전에 실시간 상호 작용을 살펴보도록 하겠습니다.

다시 말하지만, 결과를 멋진 데이터 프레임으로 렌더링하는 헬퍼 함수를 만듭니다.

#### API 호출 결과

In [18]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df_users(recommendations_df, user_id):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    #print(recommendation_list)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [user_id])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [19]:
recommendations_df_users = pd.DataFrame()
#users = users_df.sample(3).index.tolist()

for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)

recommendations_df_users

115  \
0                 There's Something About Mary (1998)   
1                            Never Been Kissed (1999)   
2                                 American Pie (1999)   
3                   10 Things I Hate About You (1999)   
4                              You've Got Mail (1998)   
5                          Shakespeare in Love (1998)   
6                          Wedding Singer, The (1998)   
7                                 Analyze This (1999)   
8        Austin Powers: The Spy Who Shagged Me (1999)   
9                     My Best Friend's Wedding (1997)   
10                                  Armageddon (1998)   
11                          As Good as It Gets (1997)   
12                             American Beauty (1999)   
13  Austin Powers: International Man of Mystery (1...   
14                                 Chasing Amy (1997)   
15                                Galaxy Quest (1999)   
16                                     Payback (1999)   
17                          Mask of Zorro, The (1998)   
18                         Grosse Pointe Blank (1997)   
19                               Runaway Bride (1999)   
20                                Notting Hill (1999)   
21                            Mickey Blue Eyes (1999)   
22                              She's All That (1999)   
23                         Blast from the Past (1999)   
24                                  Mummy, The (1999)   

                                                 518  \
0   Star Wars: Episode I - The Phantom Menace (1999)   
1                                   Ring, The (2002)   
2                                         Saw (2004)   
3                                   Game, The (1997)   
4                                    Sin City (2005)   
5                               Casino Royale (2006)   
6                            Incredibles, The (2004)   
7                               Air Force One (1997)   
8                                     Old Boy (2003)   
9                                  Zombieland (2009)   
10                                     Aliens (1986)   
11                                Constantine (2005)   
12                             Bruce Almighty (2003)   
13                            American Psycho (2000)   
14                          Kill Bill: Vol. 2 (2004)   
15                              Terminal, The (2004)   
16                   Thomas Crown Affair, The (1999)   
17                          National Treasure (2004)   
18                              28 Days Later (2002)   
19                                     Scream (1996)   
20                 10 Things I Hate About You (1999)   
21                              Sleepy Hollow (1999)   
22                         Brokeback Mountain (2005)   
23                                I Am Legend (2007)   
24                        Million Dollar Baby (2004)   

                                                  252  
0                                   Big Hero 6 (2014)  
1                           King's Speech, The (2010)  
2                                        Shrek (2001)  
3                                      WALLÂ·E (2008)  
4                                The Boss Baby (2017)  
5                                  Ratatouille (2007)  
6                             Arthur Christmas (2011)  
7                     Wolf of Wall Street, The (2013)  
8                              The Emoji Movie (2017)  
9                             Incredibles, The (2004)  
10                                Finding Nemo (2003)  
11                              The Lego Movie (2014)  
12                                   Inception (2010)  
13                                       Fargo (1996)  
14                                     Tangled (2010)  
15                                  Inside Out (2015)  
16                                   Pinocchio (1940)  
17                             Despicable Me 2 (2013)  
18  Tale of Princess Kaguya, The (Kaguyahime n

여기에서, 각 사용자마다 추천이 서로 다르다는 것을 분명히 알 수 있습니다. 이러한 결과에 캐시가 필요한 경우 먼저 모든 사용자에 대해 API 호출을 실행하고 결과를 저장하거나, 이 노트북의 뒷부분에서 다룰 배치 내보내기를 사용할 수 있습니다.

이제 항목 필터를 적용하여 장르 내에서 이러한 사용자 중 하나에 대한 추천을 확인하겠습니다.


In [20]:
def get_new_recommendations_df_by_filter(recommendations_df, user_id, filter_arn):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
        filterArn = filter_arn
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    #print(recommendation_list)
    filter_name = filter_arn.split('/')[1]
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [filter_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

특정 장르 내에서 추천하는 영화를 확인할 수 있습니다. VOD 애플리케이션 내에서 이러한 필터를 사용하여 선반(레일 또는 캐로셀이라고도 함)을 쉽게 만들 수 있습니다. 항목에 대한 정보에 따라 키워드, 연도/연대 등의 추가 정보를 필터링할 수도 있습니다.

In [21]:
recommendations_df_shelves = pd.DataFrame()
for filter_arn in meta_filter_arns:
    recommendations_df_shelves = get_new_recommendations_df_by_filter(recommendations_df_shelves, user, filter_arn)
for filter_arn in decade_filter_arns:
    recommendations_df_shelves = get_new_recommendations_df_by_filter(recommendations_df_shelves, user, filter_arn)

recommendations_df_shelves

Film-Noir  \
0                Tinker Tailor Soldier Spy (2011)   
1                                    Drive (2011)   
2                       Call Northside 777 (1948)   
3                        L.A. Confidential (1997)   
4                                 Sin City (2005)   
5                                Key Largo (1948)   
6                 Night of the Hunter, The (1955)   
7   Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)   
8                         Double Indemnity (1944)   
9                                    Gilda (1946)   
10                              White Heat (1949)   
11                   While the City Sleeps (1956)   
12                    Strangers on a Train (1951)   
13                             Renaissance (2006)   
14                    Stray Dog (Nora inu) (1949)   
15                            Dark Passage (1947)   
16                                    Fury (1936)   
17                             High Sierra (1941)   
18                        Mulholland Drive (2001)   
19                     Sorry, Wrong Number (1948)   
20                               Chinatown (1974)   
21                                  Rubber (2010)   
22                  Sweet Smell of Success (1957)   
23                            Johnny Eager (1942)   
24                 Angels with Dirty Faces (1938)   

                                              Fantasy  \
0                                        Shrek (2001)   
1                               The Lego Movie (2014)   
2                                      Tangled (2010)   
3                                   Inside Out (2015)   
4                                    Pinocchio (1940)   
5   Tale of Princess Kaguya, The (Kaguyahime no mo...   
6              Snow White and the Seven Dwarfs (1937)   
7                                  Toy Story 2 (1999)   
8                     How to Train Your Dragon (2010)   
9                         Beauty and the Beast (1991)   
10                                  Your Name. (2016)   
11                                 Toy Story 3 (2010)   
12                    Kubo and the Two Strings (2016)   
13                                      Frozen (2013)   
14                                   Toy Story (1995)   
15                                Mary Poppins (1964)   
16                       Rise of the Guardians (2012)   
17                       It's a Wonderful Life (1946)   
18           Princess Mononoke (Mononoke-hime) (1997)   
19                                    Big Fish (2003)   
20       My Neighbor Totoro (Tonari no Totoro) (1988)   
21  Star Wars: Episode VII - The Force Awakens (2015)   
22                                  Cinderella (1950)   
23  Wolf Children (Okami kodomo no ame to yuki) (2...   
24                                Corpse Bride (2005)   

                                              Western  \
0                             Django Unchained (2012)   
1   For a Few Dollars More (Per qualche dollaro in...   
2                                        Rango (2011)   
3   Once Upon a Time in the West (C'era una volta ...   
4           Butch Cassidy and the Sundance Kid (1969)   
5                            The Hateful Eight (2015)   
6                                    True Grit (2010)   
7            A Million Ways to Die in the West (2014)   
8                           Dances with Wolves (1990)   
9                                 The Beguiled (2017)   
10                                   High Noon (1952)   
11  Fistful of Dollars, A (Per un pugno di dollari...   
12                            Cowboys & Aliens (2011)   
13           Man Who Shot Liberty Valance, The (1962)   
14                                3:10 to Yuma (2007)   
15                              Salvation, The (2014)   
16                                  Stagecoach (1939)   
17                       The Magnificent Seven (2016)   
18  Good, the Bad and the Ugly, The (Buono, il bru...   
19                                Misf

다음 주제는 실시간 이벤트입니다. Personalize에는 사용자에게 표시되는 추천을 업데이트하기 위해 프로그램에서 이벤트를 수신하는 기능이 있습니다. 이 기능은 온디맨드 비디오와 같은 미디어 워크로드에서 특히 유용합니다. 온디맨드 비디오의 경우 자녀와 함께 시청하는지 아니면 혼자 시청하는지에 따라 고객의 의도가 달라질 수 있습니다.

또한 이 시스템을 통해 기록된 이벤트는 삭제 호출이 실행될 때까지 저장되며, 다음 모델을 훈련할 때 제공하는 다른 상호 작용 데이터와 함께 기록 데이터로 사용됩니다.

#### 실시간 이벤트

캠페인에 연결되는 이벤트 트래커를 만드는 것부터 시작합니다.

In [22]:
response = personalize.create_event_tracker(
    name='MovieTracker',
    datasetGroupArn=dataset_group_arn
)
print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId']
event_tracker_arn = response['eventTrackerArn']

arn:aws:personalize:us-east-1:136455442858:event-tracker/88291ed2
3b3b7d7c-652c-41b4-af91-aef026c8f335


특정 항목과 상호 작용하는 사용자를 시뮬레이션하는 코드를 만듭니다. 이 코드를 실행한 후에는 위의 결과와 다른 추천을 받게 됩니다.

먼저 실시간 이벤트 시뮬레이션을 위한 몇 가지 메서드를 생성합니다.

In [23]:
session_dict = {}

def send_movie_click(USER_ID, ITEM_ID, EVENT_TYPE):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[str(USER_ID)]
    except:
        session_dict[str(USER_ID)] = str(uuid.uuid1())
        session_ID = session_dict[str(USER_ID)]
        
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    }
    event_json = json.dumps(event)
        
    # Make Call
    
    personalize_events.put_events(
    trackingId = TRACKING_ID,
    userId= str(USER_ID),
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': str(EVENT_TYPE),
        'properties': event_json
        }]
    )

def get_new_recommendations_df_users_real_time(recommendations_df, user_id, item_id, event_type):
    # Get the artist name (header of column)
    movie_name = get_movie_by_id(item_id)
    # Interact with different movies
    print('sending event ' + event_type + ' for ' + get_movie_by_id(item_id))
    send_movie_click(USER_ID=user_id, ITEM_ID=item_id, EVENT_TYPE=event_type)
    # Get the recommendations (note you should have a base recommendation DF created before)
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        artist = get_movie_by_id(item['itemId'])
        recommendation_list.append(artist)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_name])
    # Add this dataframe to the old one
    #recommendations_df = recommendations_df.join(new_rec_DF)
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

이 시점에는 아직 실시간 이벤트를 생성하지 않았으며 코드만 설정했습니다. 실시간 이벤트 전후의 추천을 비교하기 위해, 한 명의 사용자를 선택하고 해당 사용자에 대한 원래 추천을 생성해 보겠습니다.

In [29]:
# First pick a user
user_id = user

# Get recommendations for the user
get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_list = []
for item in item_list:
    artist = get_movie_by_id(item['itemId'])
    recommendation_list.append(artist)
user_recommendations_df = pd.DataFrame(recommendation_list, columns = [user_id])
user_recommendations_df

252
0                        Big Hero 6 (2014)
1                King's Speech, The (2010)
2                             Shrek (2001)
3                           WALLÂ·E (2008)
4                     The Boss Baby (2017)
5                       Ratatouille (2007)
6          Wolf of Wall Street, The (2013)
7                  Incredibles, The (2004)
8                      Finding Nemo (2003)
9                    The Lego Movie (2014)
10                        Inception (2010)
11                            Fargo (1996)
12                          Tangled (2010)
13                       Inside Out (2015)
14                        Pinocchio (1940)
15                  Despicable Me 2 (2013)
16  Wallace & Gromit: A Close Shave (1995)
17  Snow White and the Seven Dwarfs (1937)
18                      Toy Story 2 (1999)
19               Hotel Transylvania (2012)
20                            Mulan (1998)
21                         Zootopia (2016)
22         How to Train Your Dragon (2010)
23                   Hidden Figures (2016)
24             Beauty and the Beast (1991)

실시간 이벤트를 적용하기 전의 이 사용자에 대한 추천 목록이 있습니다. 이제 사용자의 상호 작용을 시뮬레이션할 임의의 아티스트 3명을 선택하고 그에 따라 추천이 어떻게 바뀌는지 살펴보겠습니다.

In [30]:
# Next generate 3 random movies
movies = items_df.sample(3).index.tolist()

In [31]:
# Note this will take about 15 seconds to complete due to the sleeps
for movie in movies:
    user_recommendations_df = get_new_recommendations_df_users_real_time(user_recommendations_df, user_id, movie,'click')
    time.sleep(5)

sending event click for Salvador (1986)
sending event click for Die Hard (1988)
sending event click for Power/Rangers (2015)


이제 클릭 이벤트로 인해 추천이 어떻게 변경되었는지 살펴볼 수 있습니다.

In [32]:
user_recommendations_df

252  \
0                        Big Hero 6 (2014)   
1                King's Speech, The (2010)   
2                             Shrek (2001)   
3                           WALLÂ·E (2008)   
4                     The Boss Baby (2017)   
5                       Ratatouille (2007)   
6          Wolf of Wall Street, The (2013)   
7                  Incredibles, The (2004)   
8                      Finding Nemo (2003)   
9                    The Lego Movie (2014)   
10                        Inception (2010)   
11                            Fargo (1996)   
12                          Tangled (2010)   
13                       Inside Out (2015)   
14                        Pinocchio (1940)   
15                  Despicable Me 2 (2013)   
16  Wallace & Gromit: A Close Shave (1995)   
17  Snow White and the Seven Dwarfs (1937)   
18                      Toy Story 2 (1999)   
19               Hotel Transylvania (2012)   
20                            Mulan (1998)   
21                         Zootopia (2016)   
22         How to Train Your Dragon (2010)   
23                   Hidden Figures (2016)   
24             Beauty and the Beast (1991)   

                                      Salvador (1986)  \
0                        To Kill a Mockingbird (1962)   
1                                 12 Angry Men (1957)   
2                                 Citizen Kane (1941)   
3                               Lion King, The (1994)   
4                             Bonnie and Clyde (1967)   
5                                  Animal Farm (1954)   
6                                      Amadeus (1984)   
7                           King's Speech, The (2010)   
8                         Rashomon (RashÃ´mon) (1950)   
9                                Graduate, The (1967)   
10                                 Rear Window (1954)   
11                                   Cast Away (2000)   
12             Snow White and the Seven Dwarfs (1937)   
13                                The Revenant (2015)   
14                       2001: A Space Odyssey (1968)   
15                                  Big Hero 6 (2014)   
16                   Ice Age: Collision Course (2016)   
17                                Finding Dory (2016)   
18                              Cool Hand Luke (1967)   
19                                       Shrek (2001)   
20                                Finding Nemo (2003)   
21                                Modern Times (1936)   
22  Tale of Princess Kaguya, The (Kaguyahime no mo...   
23                                     Breathe (2014)   
24                                   Ring, The (1927)   

                                      Die Hard (1988)  \
0                               Lion King, The (1994)   
1                                       Snatch (2000)   
2                                    Pinocchio (1940)   
3                                        Shrek (2001)   
4                                        Fargo (1996)   
5                          Princess Bride, The (1987)   
6                                   Home Alone (1990)   
7                                    Toy Story (1995)   
8   Pokemon 4 Ever (a.k.a. PokÃ©mon 4: The Movie) ...   
9                               Monsters, Inc. (2001)   
10                                    Die Hard (1988)   
11                   Ice Age: Collision Course (2016)   
12                        Beauty and the Beast (1991)   
13                   Silence of the Lambs, The (1991)   
14                City of God (Cidade de Deus) (2002)   
15                                    Fantasia (1940)   
16                                     Dunkirk (2017)   
17                                 Rear Window (1954)   
18                                  Big Hero 6 (2014)   
19                                       Risen (2016)   
20                    Kubo and the Two Strings (2016)   
21                 In the Name of the King III (2014)   
22                                    Deadpool (

위의 셀에서 인덱스 뒤의 첫 번째 열은 사용자 개인화에서 사용자의 기본 추천이며, 그 이후의 각 열에는 실시간 이벤트를 통해 상호 작용한 아티스트의 헤더와 이 이벤트가 발생한 후의 추천이 있습니다.

이러한 동작은 크게 바뀌지 않을 수 있습니다. 이는 이 데이터 세트의 특성과 몇 번의 무작위 클릭의 효과로 인해 상대적으로 제한되기 때문입니다. 이를 더 잘 이해하기 위해 더 많은 영화를 클릭하는 경우를 시뮬레이션을 해보면, 더 뚜렷한 효과를 볼 수 있습니다.

이제 상호 작용 데이터를 기준으로 항목을 필터링할 수 있는 이벤트 필터를 살펴보겠습니다. 이 데이터 세트의 경우 가져온 데이터를 기반으로 클릭하거나 시청할 수 있지만, 사용자가 설계하는 모든 상호 작용 스키마(클릭, 속도, 좋아요, 시청, 구매 등)에 기반할 수 있습니다. VOD 쉘프의 경우 타이틀을 "Top picks for you (고객님을 위한 탑 픽)"에서 "Watch again"로 옮길 수 있습니다. 다시 보기 추천은 사용자의 현재 상호 작용을 기반으로 하지만 이미 시청한 타이틀만 추천합니다.


In [33]:
recommendations_df_events = pd.DataFrame()
for filter_arn in interaction_filter_arns:
    recommendations_df_events = get_new_recommendations_df_by_filter(recommendations_df_events, user, filter_arn)
    
recommendations_df_events

watched  \
0                               Doctor Strange (2016)   
1                 Rogue One: A Star Wars Story (2016)   
2   Star Wars: Episode VII - The Force Awakens (2015)   
3                             Incredibles, The (2004)   
4                                      Arrival (2016)   
5                               Lion King, The (1994)   
6                                                 NaN   
7                                                 NaN   
8                                                 NaN   
9                                                 NaN   
10                                                NaN   
11                                                NaN   
12                                                NaN   
13                                                NaN   
14                                                NaN   
15                                                NaN   
16                                                NaN   
17                                                NaN   
18                                                NaN   
19                                                NaN   
20                                                NaN   
21                                                NaN   
22                                                NaN   
23                                                NaN   
24                                                NaN   

                                            unwatched  
0                                     Deadpool (2016)  
1                    Guardians of the Galaxy 2 (2017)  
2                      Guardians of the Galaxy (2014)  
3                                        Logan (2017)  
4                               Thor: Ragnarok (2017)  
5                            X-Men: Apocalypse (2016)  
6                             Edge of Tomorrow (2014)  
7                                  The Martian (2015)  
8                                       Aliens (1986)  
9              Avengers: Infinity War - Part I (2018)  
10                                  Deadpool 2 (2018)  
11       Ghost in the Shell (KÃ´kaku kidÃ´tai) (1995)  
12   Star Wars: Episode I - The Phantom Menace (1999)  
13                     Avengers: Age of Ultron (2015)  
14  Pirates of the Caribbean: The Curse of the Bla...  
15                                Interstellar (2014)  
16                                    Iron Man (2008)  
17                               Black Panther (2017)  
18                          Mad Max: Fury Road (2015)  
19              Dawn of the Planet of the Apes (2014)  
20  Laputa: Castle in the Sky (TenkÃ» no shiro Rap...  
21                                       Akira (1988)  
22                                The Revenant (2015)  
23                  X-Men: Days of Future Past (2014)  
24          Batman v Superman: Dawn of Justice (2016)

이제 시청하지 않은 4개의 추천에 대해 4개의 영화를 시청하는 경우를 시뮬레이션하는 시청 이벤트를 보내겠습니다. VOD 애플리케이션에서 콘텐츠의 상당량(75%)을 시청하고 나면 이벤트를 보내도록 선택할 수 있습니다. 100% 완료 시를 기준으로 이벤트를 보내면 크레딧을 마저 보지 않은 사람을 놓칠 수 있습니다.

In [34]:
 # Get the recommendations
top_unwatched_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = userpersonalization_campaign_arn,
    userId = str(user_id),
    filterArn = filter_arn,
    numResults=4)
item_list = top_unwatched_recommendations_response['itemList']
for item in item_list:
    print('sending event watch for ' + get_movie_by_id(item['itemId']))
    send_movie_click(USER_ID=user_id, ITEM_ID=item['itemId'], EVENT_TYPE='watch')
    time.sleep(10)

sending event watch for Deadpool (2016)
sending event watch for Guardians of the Galaxy 2 (2017)
sending event watch for Guardians of the Galaxy (2014)
sending event watch for Logan (2017)


이제 이벤트 필터를 확인하여 업데이트된 시청한 추천과 시청하지 않은 추천을 확인할 수 있습니다. 

In [36]:
recommendations_df_events = pd.DataFrame()
for filter_arn in interaction_filter_arns:
    recommendations_df_events = get_new_recommendations_df_by_filter(recommendations_df_events, user, filter_arn)
    
recommendations_df_events

watched  \
0                          Logan (2017)   
1                        Arrival (2016)   
2      Guardians of the Galaxy 2 (2017)   
3                     Big Hero 6 (2014)   
4   Rogue One: A Star Wars Story (2016)   
5                 Doctor Strange (2016)   
6                                   NaN   
7                                   NaN   
8                                   NaN   
9                                   NaN   
10                                  NaN   
11                                  NaN   
12                                  NaN   
13                                  NaN   
14                                  NaN   
15                                  NaN   
16                                  NaN   
17                                  NaN   
18                                  NaN   
19                                  NaN   
20                                  NaN   
21                                  NaN   
22                                  NaN   
23                                  NaN   
24                                  NaN   

                                         unwatched  
0                         Blade Runner 2049 (2017)  
1                          Edge of Tomorrow (2014)  
2                                Deadpool 2 (2018)  
3                                      Lucy (2014)  
4                            Thor: Ragnarok (2017)  
5                          Maze Runner, The (2014)  
6            Dawn of the Planet of the Apes (2014)  
7           Avengers: Infinity War - Part I (2018)  
8                  Star Wars: The Last Jedi (2017)  
9                             Suicide Squad (2016)  
10               Captain America: Civil War (2016)  
11                                  Ant-Man (2015)  
12             Bungo Stray Dogs: Dead Apple (2018)  
13                        X-Men: Apocalypse (2016)  
14                            Black Panther (2017)  
15                  The Gracefield Incident (2015)  
16               X-Men: Days of Future Past (2014)  
17       Batman v Superman: Dawn of Justice (2016)  
18                 Sharknado 3: Oh Hell No! (2015)  
19                    The Lego Batman Movie (2017)  
20                             Ghostbusters (2016)  
21                  Avengers: Age of Ultron (2015)  
22  Batman: The Dark Knight Returns, Part 1 (2012)  
23                                  Gravity (2013)  
24                              Baby Driver (2017)

### 개인별 순위

개인화된 순위의 핵심 사용 사례는 항목 컬렉션을 가지고, 사용자에게 우선 순위 또는 가능한 관심도 순서로 렌더링하는 것입니다. VOD 애플리케이션의 경우 일부 정보(감독, 로케이션, 슈퍼히어로 프랜차이즈, 영화의 시대 배경 등)에 따라 개인화된 쉘프/레일/캐러셀을 동적으로 렌더링합니다. 이는 메타데이터에 있는 정보가 아니므로 항목 메타데이터 필터가 작동하지 않을 수 있지만, 이 정보를 사용하여 항목 목록을 생성할 수 있습니다.

이를 보여주기 위해 이전과 동일한 사용자와 임의의 항목 컬렉션을 사용합니다.

In [37]:
rerank_user = user
rerank_items = items_df.sample(25).index.tolist()

이제 입력 데이터를 보여주는 멋진 데이터 프레임을 구축합니다.

In [38]:
rerank_list = []
for item in rerank_items:
    movie = get_movie_by_id(item)
    rerank_list.append(movie)
rerank_df = pd.DataFrame(rerank_list, columns = ['Un-Ranked'])
rerank_df

Un-Ranked
0                 Thrill of It All, The (1963)
1                     Fruitvale Station (2013)
2               Ice Age 2: The Meltdown (2006)
3                  Escape from New York (1981)
4                          Pulp Fiction (1994)
5                     Hurricane Streets (1997)
6                           Marine, The (2006)
7                          The Revenant (2015)
8           Snow White and the Huntsman (2012)
9                        Into the Abyss (2011)
10  Harry Potter and the Goblet of Fire (2005)
11                      Higher Learning (1995)
12                       Wreck-It Ralph (2012)
13      Better Living Through Circuitry (1999)
14                              Yes Man (2008)
15                            Zookeeper (2011)
16                             Stigmata (1999)
17              AVP: Alien vs. Predator (2004)
18                    Gleaming the Cube (1989)
19             Midnight Meat Train, The (2008)
20                                 Hush (2016)
21                             Chocolat (1988)
22                           Affliction (1997)
23             Decalogue, The (Dekalog) (1989)
24                           Earthquake (1974)

그런 다음 개인화된 순위 API를 호출합니다.

In [39]:
# Convert user to string:
user_id = str(rerank_user)
rerank_item_list = []
for item in rerank_items:
    rerank_item_list.append(str(item))
    
# Get recommended reranking
get_recommendations_response_rerank = personalize_runtime.get_personalized_ranking(
        campaignArn = rerank_campaign_arn,
        userId = user_id,
        inputList = rerank_item_list
)

나란히 비교할 수 있도록, 순위가 다시 매겨진 항목을 원래 데이터 프레임에 두 번째 열로 추가합니다.

In [40]:
ranked_list = []
item_list = get_recommendations_response_rerank['personalizedRanking']
for item in item_list:
    movie = get_movie_by_id(item['itemId'])
    ranked_list.append(movie)
ranked_df = pd.DataFrame(ranked_list, columns = ['Re-Ranked'])
rerank_df = pd.concat([rerank_df, ranked_df], axis=1)
rerank_df

Un-Ranked  \
0                 Thrill of It All, The (1963)   
1                     Fruitvale Station (2013)   
2               Ice Age 2: The Meltdown (2006)   
3                  Escape from New York (1981)   
4                          Pulp Fiction (1994)   
5                     Hurricane Streets (1997)   
6                           Marine, The (2006)   
7                          The Revenant (2015)   
8           Snow White and the Huntsman (2012)   
9                        Into the Abyss (2011)   
10  Harry Potter and the Goblet of Fire (2005)   
11                      Higher Learning (1995)   
12                       Wreck-It Ralph (2012)   
13      Better Living Through Circuitry (1999)   
14                              Yes Man (2008)   
15                            Zookeeper (2011)   
16                             Stigmata (1999)   
17              AVP: Alien vs. Predator (2004)   
18                    Gleaming the Cube (1989)   
19             Midnight Meat Train, The (2008)   
20                                 Hush (2016)   
21                             Chocolat (1988)   
22                           Affliction (1997)   
23             Decalogue, The (Dekalog) (1989)   
24                           Earthquake (1974)   

                                     Re-Ranked  
0                          The Revenant (2015)  
1                          Pulp Fiction (1994)  
2               AVP: Alien vs. Predator (2004)  
3                        Wreck-It Ralph (2012)  
4                  Escape from New York (1981)  
5                     Fruitvale Station (2013)  
6                               Yes Man (2008)  
7                              Stigmata (1999)  
8                              Chocolat (1988)  
9                             Zookeeper (2011)  
10  Harry Potter and the Goblet of Fire (2005)  
11              Ice Age 2: The Meltdown (2006)  
12                      Higher Learning (1995)  
13                    Gleaming the Cube (1989)  
14          Snow White and the Huntsman (2012)  
15                           Earthquake (1974)  
16                       Into the Abyss (2011)  
17                Thrill of It All, The (1963)  
18                                 Hush (2016)  
19                    Hurricane Streets (1997)  
20                          Marine, The (2006)  
21      Better Living Through Circuitry (1999)  
22             Midnight Meat Train, The (2008)  
23                           Affliction (1997)  
24             Decalogue, The (Dekalog) (1989)

위에서 모델의 사용자에 대한 이해를 바탕으로 각 항목이 어떻게 다시 정렬되었는지 확인할 수 있습니다. 이 작업은 사용자에게 노출할 항목의 컬렉션(예: 프로모션 목록)이 있을 때 흔히 사용됩니다.

## 배치 추천 <a class="anchor" id="batch"></a>
[맨 위로 이동](#top)

내보낸 추천으로 이루어진 더 큰 데이터 세트가 필요한 경우가 많습니다. 최근 Amazon Personalize는 추천 컬렉션을 S3로 내보내는 수단으로서 배치 추천을 발표했습니다. 이 예에서는 HRNN 솔루션에 대해 이 작업을 수행하는 방법을 살펴보겠습니다. 배치 추천에 대한 자세한 내용은 [설명서](https://docs.aws.amazon.com/personalize/latest/dg/getting-recommendations.html#recommendations-batch)를 참조하세요. 이 기능은 모든 레시피에 적용되지만 출력 형식은 레시피마다 다릅니다.

간단한 구현의 예는 다음과 같습니다.

```python
import boto3

personalize_rec = boto3.client(service_name='personalize')

personalize_rec.create_batch_inference_job (
    solutionVersionArn = "Solution version ARN",
    jobName = "Batch job name",
    roleArn = "IAM role ARN",
    jobInput = 
       {"s3DataSource": {"path": S3 input path}},
    jobOutput = 
       {"s3DataDestination": {"path":S3 output path"}}
)
```

SDK 가져오기, 솔루션 버전 알림 및 역할 알림이 모두 결정되었습니다. 이제 입력, 출력 및 작업 이름만 정의하면 됩니다.

HRNN에 대한 입력부터 시작하겠습니다. 이 입력은 다음과 같습니다.


```JSON
{"userId": "4638"}
{"userId": "663"}
{"userId": "3384"}
```

이 경우 다음과 같은 출력이 생성됩니다.

```JSON
{"input":{"userId":"4638"}, "output": {"recommendedItems": ["296", "1", "260", "318"]}}
{"input":{"userId":"663"}, "output": {"recommendedItems": ["1393", "3793", "2701", "3826"]}}
{"input":{"userId":"3384"}, "output": {"recommendedItems": ["8368", "5989", "40815", "48780"]}}
```

출력은 JSON Lines 파일로, 한 줄에 하나씩 개별 JSON 객체로 구성됩니다. 따라서 나중에 결과를 이 형식으로 수집하기 위해서는 추가 작업이 필요합니다.

### 입력 파일 생성

배치 기능을 사용할 경우, 작업이 완료되었을 때 추천을 받을 사용자를 지정합니다. 아래의 셀은 다시 몇 명의 임의의 사용자를 선택한 후 파일을 만들고 디스크에 저장합니다. 그런 다음 S3에 업로드하여 나중에 API 호출에 사용할 수 있습니다.

In [41]:
# We will use the same users from before
users
# Write the file to disk
json_input_filename = "json_input.json"
with open(data_dir + "/" + json_input_filename, 'w') as json_input:
    for user_id in users:
        json_input.write('{"userId": "' + str(user_id) + '"}\n')

In [42]:
# Showcase the input file:
!cat $data_dir"/"$json_input_filename

{"userId": "115"}
{"userId": "518"}
{"userId": "252"}


파일을 S3에 업로드하고 나중에 사용할 수 있도록 경로를 변수로 저장합니다.

In [43]:
# Upload files to S3
boto3.Session().resource('s3').Bucket(bucket_name).Object(json_input_filename).upload_file(data_dir+"/"+json_input_filename)
s3_input_path = "s3://" + bucket_name + "/" + json_input_filename
print(s3_input_path)

s3://136455442858-us-east-1-personalizepocvod/json_input.json


배치 추천은 S3에 업로드한 파일에서 입력 데이터를 읽습니다. 마찬가지로 배치 추천은 출력을 S3의 파일에 저장합니다. 따라서 결과를 저장할 출력 경로를 정의합니다.

In [44]:
# Define the output path
s3_output_path = "s3://" + bucket_name + "/"
print(s3_output_path)

s3://136455442858-us-east-1-personalizepocvod/


이제 배치 내보내기 프로세스를 시작하는 호출을 실행하면 됩니다.

In [45]:
batchInferenceJobArn = personalize.create_batch_inference_job (
    solutionVersionArn = userpersonalization_solution_version_arn,
    jobName = "VOD-POC-Batch-Inference-Job-UserPersonalization_" + str(round(time.time()*1000)),
    roleArn = role_arn,
    jobInput = 
     {"s3DataSource": {"path": s3_input_path}},
    jobOutput = 
     {"s3DataDestination":{"path": s3_output_path}}
)
batchInferenceJobArn = batchInferenceJobArn['batchInferenceJobArn']

배치 추천 호출의 상태를 추적하기 위해 아래의 while 루프를 실행합니다. Personalize는 이 작업을 수행하기 위해 인프라를 구축해야 하므로, 작업을 완료하는 데 30분 정도 걸릴 수 있습니다. 여기서는 3명의 사용자로 이루어진 데이터 세트로 이 기능을 테스트하고 있는데, 이는 이 메커니즘의 효율적인 사용 사례라고 할 수 없습니다. 일반적으로 이 기능은 대량 처리에만 사용되며, 이 경우 높은 효율성이 명확하게 나타납니다.

In [46]:
current_time = datetime.now()
print("Import Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 6*60*60 # 6 hours
while time.time() < max_time:
    describe_dataset_inference_job_response = personalize.describe_batch_inference_job(
        batchInferenceJobArn = batchInferenceJobArn
    )
    status = describe_dataset_inference_job_response["batchInferenceJob"]['status']
    print("DatasetInferenceJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.now()
print("Import Completed on: ", current_time.strftime("%I:%M:%S %p"))

Import Started on:  10:35:52 AM
DatasetInferenceJob: CREATE PENDING
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInfer

In [47]:
s3 = boto3.client('s3')
export_name = json_input_filename + ".out"
s3.download_file(bucket_name, export_name, data_dir+"/"+export_name)

# Update DF rendering
pd.set_option('display.max_rows', 30)
with open(data_dir+"/"+export_name) as json_file:
    # Get the first line and parse it
    line = json.loads(json_file.readline())
    # Do the same for the other lines
    while line:
        # extract the user ID 
        col_header = "User: " + line['input']['userId']
        # Create a list for all the artists
        recommendation_list = []
        # Add all the entries
        for item in line['output']['recommendedItems']:
            movie = get_movie_by_id(item)
            recommendation_list.append(movie)
        if 'bulk_recommendations_df' in locals():
            new_rec_DF = pd.DataFrame(recommendation_list, columns = [col_header])
            bulk_recommendations_df = bulk_recommendations_df.join(new_rec_DF)
        else:
            bulk_recommendations_df = pd.DataFrame(recommendation_list, columns=[col_header])
        try:
            line = json.loads(json_file.readline())
        except:
            line = None
bulk_recommendations_df

User: 115  \
0                 There's Something About Mary (1998)   
1                            Never Been Kissed (1999)   
2                                 American Pie (1999)   
3                   10 Things I Hate About You (1999)   
4                              You've Got Mail (1998)   
5                          Shakespeare in Love (1998)   
6                          Wedding Singer, The (1998)   
7                                 Analyze This (1999)   
8        Austin Powers: The Spy Who Shagged Me (1999)   
9                     My Best Friend's Wedding (1997)   
10                                  Armageddon (1998)   
11                          As Good as It Gets (1997)   
12                             American Beauty (1999)   
13  Austin Powers: International Man of Mystery (1...   
14                                 Chasing Amy (1997)   
15                                Galaxy Quest (1999)   
16                                     Payback (1999)   
17                          Mask of Zorro, The (1998)   
18                         Grosse Pointe Blank (1997)   
19                               Runaway Bride (1999)   
20                                Notting Hill (1999)   
21                            Mickey Blue Eyes (1999)   
22                              She's All That (1999)   
23                         Blast from the Past (1999)   
24                                  Mummy, The (1999)   

                                           User: 518  \
0   Star Wars: Episode I - The Phantom Menace (1999)   
1                                   Ring, The (2002)   
2                                         Saw (2004)   
3                                   Game, The (1997)   
4                                    Sin City (2005)   
5                               Casino Royale (2006)   
6                            Incredibles, The (2004)   
7                               Air Force One (1997)   
8                                     Old Boy (2003)   
9                                  Zombieland (2009)   
10                                     Aliens (1986)   
11                                Constantine (2005)   
12                             Bruce Almighty (2003)   
13                            American Psycho (2000)   
14                          Kill Bill: Vol. 2 (2004)   
15                              Terminal, The (2004)   
16                   Thomas Crown Affair, The (1999)   
17                          National Treasure (2004)   
18                              28 Days Later (2002)   
19                                     Scream (1996)   
20                 10 Things I Hate About You (1999)   
21                              Sleepy Hollow (1999)   
22                         Brokeback Mountain (2005)   
23                                I Am Legend (2007)   
24                        Million Dollar Baby (2004)   

                                 User: 252  
0                        Big Hero 6 (2014)  
1                King's Speech, The (2010)  
2                             Shrek (2001)  
3                           WALLÂ·E (2008)  
4                     The Boss Baby (2017)  
5                       Ratatouille (2007)  
6          Wolf of Wall Street, The (2013)  
7                  Incredibles, The (2004)  
8                      Finding Nemo (2003)  
9                    The Lego Movie (2014)  
10                        Inception (2010)  
11                            Fargo (1996)  
12                          Tangled (2010)  
13                       Inside Out (2015)  
14                        Pinocchio (1940)  
15                  Despicable Me 2 (2013)  
16                  The Emoji Movie (2017)  
17  Wallace & Gromit: A Close Shave (1995)  
18  Snow White and the Seven Dwarfs (1937)  
19                      Toy Story 2 (1999)  
20                 Arthur Christmas (2011)  
21               Hotel Transylvania (2012)  
22                            Mulan (1998)  
23                      

## 마무리 <a class="anchor" id="wrapup"></a>
[맨 위로 이동](#top)

이를 통해 다양한 추천 및 개인화 시나리오를 처리할 수 있는 완전한 모델 컬렉션과 고객 데이터를 조작하여 서비스와 보다 효과적으로 통합할 수 있는 기술, API를 통해 오픈 소스 데이터 과학 도구를 활용하여 이 모든 것을 수행하는 방법에 대한 지식을 갖출 수 있습니다.

이러한 노트북을 가이드로 사용하여 고객과 POC를 시작할 수 있습니다. 누락된 구성 요소를 찾거나 새 접근 방식을 발견한 경우, 풀 요청을 잘라내고 이 컬렉션에서 누락된 유용한 구성 요소를 추가로 제공합니다.

이 POC를 진행하는 동안 배포한 모든 리소스를 정리해야 합니다. `06_Clean_Up_Resources.ipynb`의 리소스를 식별하고 삭제하는 방법을 보여 주는 별도의 노트북을 제공했습니다.

In [48]:
%store event_tracker_arn
%store batchInferenceJobArn

Stored 'event_tracker_arn' (str)
Stored 'batchInferenceJobArn' (str)
